In [ ]:
import numpy as np

The data is stored in `3` files.
1. `train.csv` numbers `18,304` samples and contains the training data including features and labels. 
2. `test.csv` numbers `4580` samples and contains the testing feature without labels.
3. `sample.csv` number `4580` samples and contains the testing labels without features.

Every file has table headers and ends in an empty line.

In [ ]:
DATA_DIR = r'dataset-in/'   # directory holding the data files
# names of files holding dataset
DATA_FILENAMES = {\
                  r'trainXy': r'train.csv',\
                  r'testX':   r'test.csv',\
                  r'test_y':  r'sample.csv',\
                 }
DELIMITER = r','    # used to separate values in DATA_FILENAME

In [ ]:
data = {}
for datatype, file in DATA_FILENAMES.items():
    data[datatype] = np.genfromtxt(
        fr'{DATA_DIR}{file}', delimiter=DELIMITER,
        skip_header=True, dtype=np.float64)


In [ ]:
for datatype, array in data.items():
    print(f'{datatype:8}\t{len(array):8}')

trainXy 	   18304
testX   	    4580
test_y  	    4580
